In [17]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re 


In [186]:
# Me conecto con el ecb para pedir las tasas de cambio con respecto al euro
url = ("https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html")
html = requests.get(url).content
result = BeautifulSoup(html,"lxml")

In [205]:
## obtengo las tasas de cambio y genero un DataFrame para inspeccionar que estén bien. 
country_ = result.select(".currency")
country = [ctry.text for ctry in country_]
country.insert(0,"EUR")
currency_ = result.select(".alignLeft")
currency = [curr.text for curr in currency_]
currency.insert(0,"Euro")
spot_ = result.select(".rate")
spot = [sp.text for sp in spot_]
spot.insert(0,1)
dictionary = {"Country":country,"Currency":currency,"Spot":spot}
df = pd.DataFrame(dictionary)
display(df.head(3))

,Country,Currency,Spot
0,EUR,Euro,1
1,USD,US dollar,1.1417
2,JPY,Japanese yen,128.89


In [150]:
# Me conecto con ebay para ver los productos de la seccion de "Deals"
url_2 = ("https://www.ebay.es/deals")
html_2 = requests.get(url_2).content
result = BeautifulSoup(html_2,"lxml")

In [151]:
# Descargo los nombres de los productos con sus respectivos precios. 
titulos_ = result.select(".ebayui-ellipsis-2")
titulos = [titul.text for titul in titulos_]
precios_ = result.select(".first")
precios = [prec.text for prec in precios_[1:]]

In [204]:
# Construyo el DataFrame de la seccion "Deals" en Ebay para inspeccionar que este bien.   
d = dict(zip(titulos,precios))
pr = pd.Series(d, name="Precios")
pr.index.name = "Titulos"
pr2 = pr.reset_index()
df_e = pr2
display(df_e.head(5))

,Titulos,Precios
0,Patinete eléctrico Mi Xiaomi M365 Scooter negr...,"349,00 EUR"
1,"TV 49"" LG 49UK6200 UHD 4K Smart TV","399,99 EUR"
2,Radiador Calefactor Calentador Convector Eléct...,"89,99 EUR"
3,"NUEVO Samsung Galaxy Note 9 (SM-N960F/DS) 6.4""...","659,99 EUR"
4,Jack&Jones Hombre Sudadera Invierno Chaqueta 2...,"19,99 EUR"


In [200]:
# Me propongo sacar el precio de cada producto en algunas de las monedas que me traje del ecb

# Filtro la tabla de las currencys para que solo quedarme con las que me interesan (spot)
lista = ["EUR", "USD","JPY","GBP","AUD","CAD","CHF"]
df_cur = df[df["Country"].isin(lista)][["Currency","Spot"]]
spot = df_cur["Spot"]

# Creo un diccionario donde la key sea la currency y el valor sea la respectiva tasa de cambio 
cur_spot = dict(zip(df_cur["Currency"],df_cur["Spot"]))


In [201]:
# Convierto los precios en floats, para poder eventualmente multiplicarlo por la tasa de cambio. 
_ = [re.findall("[^A-Z.]",precio)for precio in precios]
_ = [("".join(prec)) for prec in _]
price_list = [float(prec_f.replace(",","."))  for prec_f in _]

In [211]:
# Itero sobre el precio de cada producto para calcular su precio en las distinas monedas.
lista = []
for precio in price_list:
    diccionario = {}
    for currency,spot in cur_spot.items():
        diccionario[currency] = precio*float(spot)
    lista.append(diccionario)

# Creo un diccionario con el titulo del producto y sus respectivos precios en cada moneda y luego genero el DF. 
tit_precios = dict(zip(titulos,lista))
final = pd.DataFrame(tit_precios).transpose().reset_index()
final = final.rename(columns={"index":"Products"})
display(final.head(5))

,Products,Australian dollar,Canadian dollar,Euro,Japanese yen,Pound sterling,Swiss franc,US dollar
0,Patinete eléctrico Mi Xiaomi M365 Scooter negr...,552.81600,521.964400,349.00,44982.6100,306.666300,398.627800,398.453300
1,"TV 49"" LG 49UK6200 UHD 4K Smart TV",633.58416,598.225044,399.99,51554.7111,351.471213,456.868578,456.668583
2,Radiador Calefactor Calentador Convector Eléct...,142.54416,134.589044,89.99,11598.8111,79.074213,102.786578,102.741583
3,"NUEVO Samsung Galaxy Note 9 (SM-N960F/DS) 6.4""...",1045.42416,987.081044,659.99,85066.1111,579.933213,753.840578,753.510583
4,Jack&Jones Hombre Sudadera Invierno Chaqueta 2...,31.66416,29.897044,19.99,2576.5111,17.565213,22.832578,22.822583


In [212]:
data = final.to_csv("Output/product_prices.csv",index=False)
